# HW6

In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Activation, MaxPooling2D, Flatten

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = Sequential()
model.add(Dense(4, input_shape=(6,), activation='tanh'))
model.add(Dense(4, activation='tanh'))
model.add(Dense(1, activation='tanh'))

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn import datasets
from sklearn.model_selection import train_test_split

In [4]:
import pandas as pd
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [6]:
!pip install mlxtend

    100% |████████████████████████████████| 1.3MB 6.1MB/s eta 0:00:01


Read all EMNIST test and train data

In [7]:
from mlxtend.data import loadlocal_mnist

X_train, y_train = loadlocal_mnist(
        images_path="./emnist-balanced-train-images-idx3-ubyte", 
        labels_path="./emnist-balanced-train-labels-idx1-ubyte")



In [8]:
from mlxtend.data import loadlocal_mnist

X_test, y_test = loadlocal_mnist(
        images_path="./emnist-balanced-test-images-idx3-ubyte", 
        labels_path="./emnist-balanced-test-labels-idx1-ubyte")

Reading mapping of the labels and convert ASCII values to chars

In [9]:
mapping = []
with open('./emnist-balanced-mapping.txt') as f:
    for line in f:
        mapping.append(chr(int(line.split()[1])))

Converting data to numpy arrays and normalize images to the interval [0,1]

In [10]:
X_train = np.array(X_train) / 255
y_train = np.array(y_train)
X_test = np.array(X_test) / 255
y_test = np.array(y_test)

In [11]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense

def build_model(nb_classes, nb_filters, kernel_size, pool_size, input_shape):
    model = Sequential()
    model.add(Convolution2D(int(nb_filters / 2), kernel_size, padding='valid',
                            input_shape=input_shape, activation='relu',
                            kernel_initializer='he_normal', data_format = 'channels_first'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Convolution2D(nb_filters, kernel_size, activation='relu', 
                            kernel_initializer='he_normal', data_format = 'channels_first'))
    model.add(MaxPooling2D(pool_size=pool_size))
    
    model.add(Flatten())
    model.add(Dense(250, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(125, activation='relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax', kernel_initializer='he_normal'))

    model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])
    print(model.summary())
    
    return model

Setting the parameters to be used with our CNN and preprocess the data a bit further in order for it to be in the necessary shape.

len(mapping) is for number of classes in the train set 
nb_filters is number of convolutional filters
our kernel size is (5,5) and pooling area is (2,2)
We reshaped the image size as 1,28,28 to be used in a Convolutional Neural Network
We need one-hot encoding of the label arrays

The Keras library offers a function called to_categorical() that you can use to one hot encode integer data.

In [12]:
from keras.utils import np_utils


nb_classes = len(mapping)
nb_filters = 32
kernel_size = (5, 5) 
pool_size = (2, 2)
input_shape = (1, 28, 28)

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)


y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

Building our CNN model and fit it with the EMNIST training data.

In [13]:
model = build_model(nb_classes, nb_filters, kernel_size, pool_size, input_shape)
model.fit(X_train, y_train, batch_size=128, epochs=12)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 24, 24)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 12, 24)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 8, 20)         6432      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 4, 20)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 250)               320250    
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
__________

Evaluating our CNN with the test data

In [14]:
test_eval = model.evaluate(X_test, y_test)
print()
print('    Test set loss:', test_eval[0])
print('Test set accuracy:', test_eval[1])

18800/18800 [==============================] - 60s    

    Test set loss: 0.39020500284560183
Test set accuracy: 0.8661170212765957


I trained it twice but it can be still trained more to have better accuracy.